In [1]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
import tarfile
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#Load tar.gz file into Python
#We are using day 1 of 7

tar = tarfile.open("/Users/xiwu/Desktop/Math-M148-data-january2023/hitdata7days_0.tar.gz","r:gz")

In [4]:
#Extracting filename from TarInfo object
#Gives name of file - can be regular data file or directory 

tar.next()

tn = tar.next()
print(tn.name)

tn1 = tar.next()
print(tn1.name)

tn2 = tar.next()
print(tn2.name)

tn3 = tar.next()
print(tn3.name)

tn4 = tar.next()
print(tn4.name)

tn5 = tar.next()
print(tn5.name)

tn6 = tar.next()
print(tn6.name)

tn7 = tar.next()
print(tn7.name)

tn8 = tar.next()
print(tn8.name)

hitdata7days/visitday=10/0017_part_00.parquet
hitdata7days/visitday=10/0050_part_00.parquet
hitdata7days/visitday=10/0049_part_00.parquet
hitdata7days/visitday=10/0033_part_00.parquet
hitdata7days/visitday=10/0001_part_00.parquet
hitdata7days/visitday=10/0018_part_00.parquet
hitdata7days/visitday=10/0083_part_00.parquet
hitdata7days/visitday=10/0065_part_00.parquet
hitdata7days/visitday=10/0117_part_00.parquet


In [5]:
#Reading info from parquet file
#Gives info on data types of columns in 1 parquet file
#Check if TarInfo is a file or a directory

print(tn.isreg())
print(tn.isdir())

pq.read_schema(tn.name)

True
False


hitdatahistorymkey: int64
filename: string
linenumber: int32
brandcode: string
visitoridhigh: decimal128(20, 0)
visitoridlow: decimal128(20, 0)
visitnumber: decimal128(10, 0)
visitdatetime: timestamp[ns]
visitdate: date32[day]
visitmonth: int32
fiscalyear: int16
fiscalmonthnumber: int16
fiscalweeknumber: int16
hit_time_gmt: decimal128(11, 0)
service: string
acceptlanguage: string
eventlist: string
homepage: string
ip: string
pageevent: decimal128(3, 0)
pageeventvar1: string
pageeventvar2: string
pagetype: string
pageurl: string
pagename: string
productlist: string
userserver: string
channel: string
sitesection: string
category: string
subcategory: string
subcategory2: string
pagecategory: string
searchterms: string
searchresults: string
refinementtype: string
refinementattribute: string
myaccountengagement: string
formanalysis: string
emailsubscriptionadd: string
emailsubscriptionremove: string
linklocation: string
navigationlinks: string
searchtype: string
businessunit: string
categor

In [6]:
#Read in parquet file as pandas DataFrame
#Turns parquet file into data frame

df = pd.read_parquet(tn.name)

df1 = pd.read_parquet(tn1.name)

df2 = pd.read_parquet(tn2.name)

df3 = pd.read_parquet(tn3.name)

df4 = pd.read_parquet(tn4.name)

df5 = pd.read_parquet(tn5.name)

df6 = pd.read_parquet(tn6.name)

df7 = pd.read_parquet(tn7.name)

df8 = pd.read_parquet(tn8.name)

df = pd.concat([df, df1, df2, df3, df4, df5, df6, df7, df8])

In [7]:
df.head

<bound method NDFrame.head of         hitdatahistorymkey                                  filename  \
0              18598956716  hit_data_fingerhutcomprod_2022-12-10.tsv   
1              18602636734  hit_data_fingerhutcomprod_2022-12-10.tsv   
2              18604605529  hit_data_fingerhutcomprod_2022-12-10.tsv   
3              18603032733  hit_data_fingerhutcomprod_2022-12-10.tsv   
4              18603492850  hit_data_fingerhutcomprod_2022-12-10.tsv   
...                    ...                                       ...   
139071         18604020670  hit_data_fingerhutcomprod_2022-12-10.tsv   
139072         18604019350  hit_data_fingerhutcomprod_2022-12-10.tsv   
139073         18606546317  hit_data_fingerhutcomprod_2022-12-10.tsv   
139074         18606570125  hit_data_fingerhutcomprod_2022-12-10.tsv   
139075         18602733262  hit_data_fingerhutcomprod_2022-12-10.tsv   

        linenumber brandcode        visitoridhigh         visitoridlow  \
0            14603       FDM  2

In [8]:
#function useful to see variable value counts of a specific column... 
#i.e. for device type how many tablets, desktops, etc...

def seeValues(data, category):

    counts = pd.DataFrame(data.groupby([category])[category].count())

    counts.rename(columns={category: 'counts'}, inplace=True,)

    counts = counts.sort_values(['counts'], ascending=[0])

    graph = counts.reset_index()

    return graph

In [9]:
#get master list of visit ids that resulted in a purchase

ids_purchase = (df[df['checkoutthankyouflag'] == 1])['visitid'].drop_duplicates()

ids_purchase = ids_purchase.tolist()

In [10]:
#create DF w only beginning hit of the visit

#filter by checking when sevrer call time matches visit start time

df_first_visit = df[df['hit_time_gmt']==df['visitstarttimegmt']]

In [11]:
#oldest visits first

df_first_visit = df_first_visit.sort_values(by='hit_time_gmt', inplace = False)

In [12]:
#lastly, grab first click of each user, because there are some duplicates

df_first_visit = df_first_visit.drop_duplicates(subset=['visitid'], keep = "first", inplace = False)

In [13]:
#number of visits 

len(df_first_visit)

46657

In [14]:
#number of visits that resulted in a purchase

len(ids_purchase)

3714

In [15]:
#add target variable - assings value 1 if visit resulted in a purchase; 0 if not

target = list()

for ids in df_first_visit['visitid']:
    
    if ids in ids_purchase:
        
        target.append("1")
        
    else:
        
        target.append("0")
        
df_first_visit.insert(0, "target", target)

In [16]:
df_first_visit.head(3) 

,target,hitdatahistorymkey,filename,linenumber,brandcode,visitoridhigh,visitoridlow,visitnumber,visitdatetime,visitdate,visitmonth,fiscalyear,fiscalmonthnumber,fiscalweeknumber,hit_time_gmt,service,acceptlanguage,eventlist,homepage,ip,pageevent,pageeventvar1,pageeventvar2,pagetype,pageurl,pagename,productlist,userserver,channel,sitesection,category,subcategory,subcategory2,pagecategory,searchterms,searchresults,refinementtype,refinementattribute,myaccountengagement,formanalysis,emailsubscriptionadd,emailsubscriptionremove,linklocation,navigationlinks,searchtype,businessunit,categoryxid,alternative,rbtused,resulttype,refinementattribute2,refinementattribute3,refinementattribute4,refinementattribute5,refinementattribute6,refinementattribute7,pagetypemerch,prop29,flashcreative,flashcreativeengagement,pdpengagement,prop33,prop34,prop35,prop36,prop37,prop38,prop39,prop40,prop41,language,prop43,prop44,prop45,prop46,prop47,prop48,prop49,prop50,ordernumber,state,useragent,zip,searchengine,excludehit,hier1,hier2,hier3,hier4,hier5,browser,postbrowserheight,postbrowserwidth,postcookies,postjavaenabled,postpersistentcookie,color,connectiontype,country,domain,postttimeinfo,javascript,language_0001,os,plugins,resolution,lasthittimegmt,firsthittimegmt,visitstarttimegmt,lastpurchasetimegmt,lastpurchasenum,firsthitpagename,visitstartpagename,visitsearchengine,visitpagenum,prevpage,geocity,geocountry,georegion,duplicatepurchase,newvisit,dailyvisitor,hourlyvisitor,monthlyvisitor,yearlyvisitor,postcampaign,sitesection_0001,category_0001,subcategory_0001,subcategory2_0001,internaltrackingcode,searchterms_0001,productfindingmethods,paymentmethod,shippingmethod,ordernumber_0001,warrantiespresented,warrantycategory,crosssoldproduct,crosssoldinfluence,crossselltype,evar16,promocode,formanalysis2,calculatorengagement,bazaarvoice,catalogid,customerconnection,evar23,evar24,evar25,evar26,scene7,evar28,addonsbuytogether,addonsymal,mvtparameters,evar34,evar35,evar36,evar37,evar38,evar39,language_0002,freshstartcomplete,opentobuy,institutionid,cdeincreasepayment,cdedeferredoffer,cdedspomodel,cdeemailoptin,evar48,paidsearchrawquery,evar50,post_evar1,post_evar2,post_evar3,post_evar4,post_evar5,post_evar6,post_evar7,post_evar8,post_evar9,post_evar10,post_evar11,post_evar12,post_evar13,post_evar14,post_evar15,post_evar16,post_evar17,post_evar18,post_evar19,post_evar20,post_evar21,post_evar22,post_evar23,post_evar24,post_evar25,post_evar26,post_evar27,post_evar28,post_evar29,post_evar30,post_evar33,post_evar34,post_evar35,post_evar36,post_evar37,post_evar38,post_evar39,post_evar40,post_evar41,post_evar42,post_evar43,post_evar44,post_evar45,post_evar46,post_evar47,post_evar48,post_evar49,post_evar50,clickaction,clickactiontype,clickcontext,clickcontexttype,clicksourceid,clicktag,visitid,cookieid,checkoutthankyouflag,evar51,evar52,evar53,evar54,evar55,evar56,evar57,evar58,evar59,evar60,evar61,evar62,evar63,evar64,evar65,evar66,evar67,evar68,evar69,evar70,evar71,evar72,evar73,evar74,evar75,evar76,evar77,evar78,evar79,evar80,evar81,evar82,evar83,evar84,evar85,evar86,evar87,evar88,evar89,evar90,evar91,evar92,evar93,evar94,evar95,evar96,evar97,evar98,evar99,evar100,initialloaddate,updatedloaddate,devicetype
40243,0,18598982044,hit_data_fingerhutcomprod_2022-12-10.tsv,39955,FDM,6219599642475497456,6803658491431386667,1,2022-12-10 00:00:00,2022-12-10,12,2022,12,50,1670652000,pe,en-US,"111,145,155,171,10038,10039,10049,10051,10096",U,94.176.80.21,10,None,no link_name,None,https://www.fingerhut.com/product/NOBV1,pdp:Los Angeles Pop Art Womenâ€™s Word-Art Pul...,None,None,None,None,None,None,None,product detail page,None,None,None,None,None,None,None,https://www.fingerhut.com/product/NOBV1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6a5d0c4252424465bf6a0d0b738ebc90,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1...,None,0,0,None,No

In [ ]:
df_first_visit['eventlist'] = df_first_visit['eventlist'].str.split(',').str.len()
df_first_visit. columns

In [ ]:
dft = df_first_visit['eventlist']
dft

In [ ]:
#data frame for percent of non-null values of each variable for PURCHASE VISITS

purchase_percent = pd.DataFrame()

df_first_visit_purchases = df_first_visit[df_first_visit['visitid'].isin(ids_purchase)]

In [ ]:
for names in df_first_visit_purchases.columns:
    
    purchase_percent[names] = [100 * (sum(seeValues(df_first_visit_purchases, names)['counts'])) / len(df_first_visit_purchases)]

In [ ]:
purchase_percent

In [ ]:
#histogram of variable frequency for PURCHASE VISITS

fig, ax = plt.subplots(figsize =(10, 7))

ax.hist(purchase_percent.iloc[0,:], bins = range(0,100,5))

plt.xlabel("Percent of Observations")
plt.ylabel("Number of Variables")

In [ ]:
#Based on histogram distribution, we store features with a 5% or more values

features = list()

for names in purchase_percent.columns:
    
    if (purchase_percent[names][0]) > 5:

        features.append(names)

In [ ]:
len(features)

In [ ]:
#subset data frame with selected features

df_first_subset = df_first_visit[features]

In [ ]:
df_first_subset.head(1)

In [ ]:
# manually remove redundant variables, useless variables. or variables we did not want to weigh highly - such as date

removed = ['hitdatahistorymkey', 'filename', 'linenumber', 'visitoridhigh', 'visitoridlow',
          'visitdatetime', 'visitdate', 'visitmonth', 'visitid', 'fiscalyear', 'fiscalweeknumber', 'fiscalmonthnumber',
          'visitstarttimegmt', 'firsthittimegmt', 'lasthittimegmt', 'initialloaddate', 'updatedloaddate',
         'cookieid']

In [ ]:
for variables in df_first_subset.columns:
    
    if variables in removed:
        
        features.remove(variables) 

In [ ]:
len(features)

In [ ]:
#subset dataframe with manually chosen features removed

df_second_subset = df_first_subset[features]

In [ ]:
df_second_subset.head(1)

In [ ]:
#replace n/a or type none values with a string "None"
#the absence of a value is sometimes useful information

df_second_subset = df_second_subset.fillna("None")

In [ ]:
#variance threshold; 3rd subset method
#get rid of variables with very low variance i.e. same value every time

ord_enc = OrdinalEncoder()

variance_train = df_second_subset.loc[:, df_second_subset.columns != 'target']

variance_transformed = ord_enc.fit_transform(variance_train)

In [ ]:
#using a threshold of 0.01 would mean dropping the column where 
#99% of the values are similar

var_thr = VarianceThreshold(threshold = 0.01) 

var_thr.fit(variance_transformed)

In [ ]:
#create new subsetted data after variance threshold

df_third_subset = (variance_train.loc[:,var_thr.get_support()])

df_third_subset.insert(0, "target", target)

In [ ]:
features = df_third_subset.columns.to_list()

In [ ]:
len(features) 

In [ ]:
#4th subset - random forest feature importance

#Class weights are balanced

clf = RandomForestClassifier(class_weight='balanced')

In [ ]:
#Create test and train sets 

X_train, X_test, y_train, y_test = train_test_split(df_third_subset.iloc[:,1:], df_third_subset['target'], test_size=0.3)

In [ ]:
#Transform categorical varaibles to become Ordinal

ord_enc = OrdinalEncoder()

X_train = ord_enc.fit_transform(X_train)

y_train = ord_enc.fit_transform(y_train.to_numpy().reshape(-1,1))

X_test = ord_enc.fit_transform(X_test)

y_test = ord_enc.fit_transform(y_test.to_numpy().reshape(-1,1))

In [ ]:
#fit the model

clf.fit(X_train, y_train)

In [ ]:
#return the feature importances

clf.feature_importances_

In [ ]:
#accuracy of decision if tree if we were to use the whole data set 

clf.score(X_test, y_test)

In [ ]:
#print out the features and importances

feat_list = []

total_importance = 0
# Print the name and gini importance of each feature
for feature in zip(df_third_subset.iloc[:,1:].columns, clf.feature_importances_):
    feat_list.append(feature)
    total_importance += feature[1]

included_feats = []
# Print the name and gini importance of each feature
for feature in zip(df_third_subset.iloc[:,1:].columns, clf.feature_importances_):
    if feature[1] > .05:
        included_feats.append(feature[0])

print('\n',"Cumulative Importance =", total_importance)

# create DataFrame using data
df_imp = pd.DataFrame(feat_list, columns =['FEATURE', 'IMPORTANCE']).sort_values(by='IMPORTANCE', ascending=False)
df_imp['CUMSUM'] = df_imp['IMPORTANCE'].cumsum()

df_imp.head(50)




## correlation map

In [ ]:
feat_list

In [ ]:
features_extract = [fest[0] for fest in feat_list]
len(features_extract)

In [17]:
#test = df_first_visit[features_extract]
test = df_first_visit
print(len(test.columns))
test.columns
test


284


,target,hitdatahistorymkey,filename,linenumber,brandcode,visitoridhigh,visitoridlow,visitnumber,visitdatetime,visitdate,visitmonth,fiscalyear,fiscalmonthnumber,fiscalweeknumber,hit_time_gmt,service,acceptlanguage,eventlist,homepage,ip,pageevent,pageeventvar1,pageeventvar2,pagetype,pageurl,pagename,productlist,userserver,channel,sitesection,category,subcategory,subcategory2,pagecategory,searchterms,searchresults,refinementtype,refinementattribute,myaccountengagement,formanalysis,emailsubscriptionadd,emailsubscriptionremove,linklocation,navigationlinks,searchtype,businessunit,categoryxid,alternative,rbtused,resulttype,refinementattribute2,refinementattribute3,refinementattribute4,refinementattribute5,refinementattribute6,refinementattribute7,pagetypemerch,prop29,flashcreative,flashcreativeengagement,pdpengagement,prop33,prop34,prop35,prop36,prop37,prop38,prop39,prop40,prop41,language,prop43,prop44,prop45,prop46,prop47,prop48,prop49,prop50,ordernumber,state,useragent,zip,searchengine,excludehit,hier1,hier2,hier3,hier4,hier5,browser,postbrowserheight,postbrowserwidth,postcookies,postjavaenabled,postpersistentcookie,color,connectiontype,country,domain,postttimeinfo,javascript,language_0001,os,plugins,resolution,lasthittimegmt,firsthittimegmt,visitstarttimegmt,lastpurchasetimegmt,lastpurchasenum,firsthitpagename,visitstartpagename,visitsearchengine,visitpagenum,prevpage,geocity,geocountry,georegion,duplicatepurchase,newvisit,dailyvisitor,hourlyvisitor,monthlyvisitor,yearlyvisitor,postcampaign,sitesection_0001,category_0001,subcategory_0001,subcategory2_0001,internaltrackingcode,searchterms_0001,productfindingmethods,paymentmethod,shippingmethod,ordernumber_0001,warrantiespresented,warrantycategory,crosssoldproduct,crosssoldinfluence,crossselltype,evar16,promocode,formanalysis2,calculatorengagement,bazaarvoice,catalogid,customerconnection,evar23,evar24,evar25,evar26,scene7,evar28,addonsbuytogether,addonsymal,mvtparameters,evar34,evar35,evar36,evar37,evar38,evar39,language_0002,freshstartcomplete,opentobuy,institutionid,cdeincreasepayment,cdedeferredoffer,cdedspomodel,cdeemailoptin,evar48,paidsearchrawquery,evar50,post_evar1,post_evar2,post_evar3,post_evar4,post_evar5,post_evar6,post_evar7,post_evar8,post_evar9,post_evar10,post_evar11,post_evar12,post_evar13,post_evar14,post_evar15,post_evar16,post_evar17,post_evar18,post_evar19,post_evar20,post_evar21,post_evar22,post_evar23,post_evar24,post_evar25,post_evar26,post_evar27,post_evar28,post_evar29,post_evar30,post_evar33,post_evar34,post_evar35,post_evar36,post_evar37,post_evar38,post_evar39,post_evar40,post_evar41,post_evar42,post_evar43,post_evar44,post_evar45,post_evar46,post_evar47,post_evar48,post_evar49,post_evar50,clickaction,clickactiontype,clickcontext,clickcontexttype,clicksourceid,clicktag,visitid,cookieid,checkoutthankyouflag,evar51,evar52,evar53,evar54,evar55,evar56,evar57,evar58,evar59,evar60,evar61,evar62,evar63,evar64,evar65,evar66,evar67,evar68,evar69,evar70,evar71,evar72,evar73,evar74,evar75,evar76,evar77,evar78,evar79,evar80,evar81,evar82,evar83,evar84,evar85,evar86,evar87,evar88,evar89,evar90,evar91,evar92,evar93,evar94,evar95,evar96,evar97,evar98,evar99,evar100,initialloaddate,updatedloaddate,devicetype
40243,0,18598982044,hit_data_fingerhutcomprod_2022-12-10.tsv,39955,FDM,6219599642475497456,6803658491431386667,1,2022-12-10 00:00:00,2022-12-10,12,2022,12,50,1670652000,pe,en-US,"111,145,155,171,10038,10039,10049,10051,10096",U,94.176.80.21,10,None,no link_name,None,https://www.fingerhut.com/product/NOBV1,pdp:Los Angeles Pop Art Womenâ€™s Word-Art Pul...,None,None,None,None,None,None,None,product detail page,None,None,None,None,None,None,None,https://www.fingerhut.com/product/NOBV1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6a5d0c4252424465bf6a0d0b738ebc90,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1...,None,0,0,None,No

In [79]:
# convert all columns to numeric type
test = test.apply(pd.to_numeric, errors='coerce')

# drop columns with NaN values
test2 = test.dropna(axis=1)

# print the resulting DataFrame
test2

len(test2.columns)

KeyboardInterrupt: 

In [ ]:

# select columns with numeric values
numeric_cols = test.select_dtypes(include='number').columns
print('Numeric columns:', numeric_cols)

# select columns with string values
string_cols = test.select_dtypes(include='object').columns
print('String columns:', string_cols)


In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = test2.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds, annot_kws={"fontsize":5})
plt.tight_layout()

In [ ]:
# Set the correlation threshold
corr_threshold = 0.8

# Filter the correlation matrix
cor_filtered = cor.abs()[cor.abs() > corr_threshold]

# Get the most heavily correlated variables
corr_vars = cor_filtered.unstack().sort_values(ascending=False).drop_duplicates().index[:10]

# Create a correlation frame
corr_frame = pd.DataFrame(index=corr_vars, columns=['Correlation Coefficient'])
for var1, var2 in corr_vars:
    corr_val = cor.loc[var1, var2]
    corr_frame.loc[(var1, var2), 'Correlation Coefficient'] = corr_val

# Print the correlation frame
print(corr_frame)

## convert visit date time to localized time(numeric value)

In [30]:
vt = pd.DataFrame(test['visitdatetime'])
vt

,visitdatetime
40243,2022-12-10 00:00:00
14640,2022-12-10 00:00:11
62244,2022-12-10 00:00:19
97446,2022-12-10 00:00:20
122350,2022-12-10 00:00:24
...,...
60417,2022-12-10 23:59:46
79229,2022-12-10 23:59:49
46180,2022-12-10 23:59:49
42948,2022-12-10 23:59:55


In [48]:
# # Convert the datetime to localized time (numeric value)
# vt['localtime'] = vt['visitdatetime'].apply(lambda x: x.timestamp())

# # Check if 'localtime' values are numeric
# is_numeric = pd.to_numeric(vt['localtime'], errors='coerce').notnull().all()


# print(is_numeric)
# vt
test # length = 46657


# copy
test2 = test


test2['georegion']

# Delect rows based on inverse of column values
#df1 = test[~(test['georegion'].str.isdigit())].index 
#print(df1)
#test.drop(df1, inplace = True)
#len(test)

unique_names = test2['georegion'].unique()
print(unique_names)

['ca' 'ga' 'mi' 'sc' 'il' 'de' 'md' 'ny' 'me' 'ri' 'va' 'ut' 'pa' 'ia'
 'tx' 'mo' 'tn' 'az' 'ky' 'al' 'nd' 'vt' 'nj' 'fl' 'oh' 'wi' 'nc' 'wv'
 'wa' 'or' 'nv' 'ct' 'in' 'la' 'qc' 'nm' 'co' 'ms' 'ar' 'ks' 'ne' 'mn'
 'hi' 'id' 'dc' 'ma' 'sd' '13' 'mt' 'bcn' 'ok' 'no region' 'mh' 'nh' 'son'
 '12' 'sp' 'wy' 'on' '00' 'pb' 'ak' 'up' 'z' 'tg' '16' '01' 'ceb' 'bc'
 'sfo' 'ab' 'df' 'np' 'pue' 'nb' 'chh' 'dk' 'dl' 'ban' '03' 'nl' 'ns'
 'cmx' '?' 'cav' '8' 'ch']


In [90]:
import pandas as pd
from dateutil import tz
import pytz



# convert state code as standard location code
state_tz_dict = {
    'ca': 'America/Los_Angeles',
    'ga': 'America/New_York',
    'mi': 'America/Detroit',
    'sc': 'America/New_York',
    'il': 'America/Chicago',
    'de': 'Europe/Berlin',
    'md': 'America/New_York',
    'ny': 'America/New_York',
    'me': 'America/New_York',
    'ri': 'America/New_York',
    'va': 'America/New_York',
    'ut': 'America/Denver',
    'pa': 'America/New_York',
    'ia': 'America/Chicago',
    'tx': 'America/Chicago',
    'mo': 'America/Chicago',
    'tn': 'America/New_York',
    'az': 'America/Phoenix',
    'ky': 'America/New_York',
    'al': 'America/Chicago',
    'nd': 'America/Chicago',
    'vt': 'America/New_York',
    'nj': 'America/New_York',
    'fl': 'America/New_York',
    'oh': 'America/New_York',
    'wi': 'America/Chicago',
    'nc': 'America/New_York',
    'wv': 'America/New_York',
    'wa': 'America/Los_Angeles',
    'or': 'America/Los_Angeles',
    'nv': 'America/Los_Angeles',
    'ct': 'America/New_York',
    'in': 'America/Indiana/Indianapolis',
    'la': 'America/Chicago',
    'qc': 'America/Montreal',
    'nm': 'America/Denver',
    'co': 'America/Denver',
    'ms': 'America/Chicago',
    'ar': 'America/Chicago',
    'ks': 'America/Chicago',
    'ne': 'America/Chicago',
    'mn': 'America/Chicago',
    'hi': 'Pacific/Honolulu',
    'id': 'America/Boise',
    'dc': 'America/New_York',
    'ma': 'America/New_York',
    'sd': 'America/Chicago',
    'mt': 'America/Denver',
    'bcn': 'Asia/Shanghai',
    'ok': 'America/Chicago',
    'no region': None,
    'mh': 'Pacific/Majuro',
    'nh': 'America/New_York',
    'son': None,
    '12': None,
    'sp': 'America/Sao_Paulo',
    'wy': 'America/Denver',
    'on': 'America/Toronto',
    '00': None,
    'pb': 'America/Recife',
    'ak': 'America/Anchorage',
    'up': 'Asia/Kolkata',
    'z': None,
    'tg': 'Asia/Dushanbe',
    '16': None,
    '01': None,
    'ceb': 'Asia/Manila',
    'bc': 'America/Vancouver',
    'sfo': 'America/Los_Angeles',
    'ab': 'America/Edmonton',
    'df': 'America/Mexico_City',
    'np': 'Asia/Kathmandu',
    'pue': 'America/Mexico_City',
    'nb': 'America/Halifax',
    'chh': 'Asia/Shanghai',
    'dk': 'Asia/Shanghai',
    'dl': 'Asia/Shanghai',
    '13': None,
    'ban':'Asia/Tehran',
    'nl': 'America/St_Johns',
    'ns': 'America/Halifax',
    'cmx': 'America/Mexico_City',
    '?': None,
    'cav': 'Asia/Shanghai',
    '8': None,
    'ch': 'Asia/Shanghai'
}

# Function to convert a geocountry and georegion into a timezone object
def get_timezone(row):
    
    country = row["geocountry"]
    region = row["georegion"]
    
#     if country.lower() == "usa":
#         country = "America"
#     elif not country.isupper():
#         country = country.title()
#     if not region.isupper():
#         region = region.upper()

    if region.isdigit() or region.isnumeric():
        return None
    
        
    tz_string = state_tz_dict[region]
    # Create a timezone string in "Area/Location" format
    #tz_string = country + '/' + region

   
    # Use dateutil.tz.gettz() to get timezone object
    #timezone = tz.gettz(tz_string)
    
    #return timezone
    return tz_string

# Function to convert a datetime to a localized time based on the timezone
def localize_time(row):
    
    if row['timezone'] is None:
        localized_time = 'NA'
        return localized_time
    else:
        # create a timezone object from the timezone string
        timezone = pytz.timezone(row['timezone'])
    
    # convert the datetime string to a datetime object in the timezone
    local_time = pd.to_datetime(row['visitdatetime']).tz_localize('UTC').tz_convert(timezone)
    
    # extract the hour component from the localized time
    hour = local_time.hour
    
    return hour
        
    
    
    
   
    # Convert the datetime to a timezone-aware datetime object
    
    #dt = pd.to_datetime(row['visitdatetime'], utc=True).replace(tzinfo=local_time)
    
    
    # Convert the datetime object to a localized time (numeric value)

    #localized_time = dt.strftime('"%m/%d/%Y %I:%M %p"')
    
    #return localized_time


# Convert the geocountry and georegion to a timezone object and store in a new column 'timezone'
vt['timezone'] = test.apply(get_timezone, axis=1)

# Convert the datetime to a localized time based on the timezone and store in a new column 'localized_time'
vt['localized_time'] = vt.apply(localize_time, axis=1)


# Print the DataFrame
print(vt)


## just store the hours


             visitdatetime             timezone localized_time
40243  2022-12-10 00:00:00  America/Los_Angeles             16
14640  2022-12-10 00:00:11     America/New_York             19
62244  2022-12-10 00:00:19      America/Detroit             19
97446  2022-12-10 00:00:20     America/New_York             19
122350 2022-12-10 00:00:24      America/Chicago             18
...                    ...                  ...            ...
60417  2022-12-10 23:59:46      America/Detroit             18
79229  2022-12-10 23:59:49     America/New_York             18
46180  2022-12-10 23:59:49  America/Los_Angeles             15
42948  2022-12-10 23:59:55     America/New_York             18
135307 2022-12-10 23:59:57  America/Los_Angeles             15

[46657 rows x 3 columns]


In [92]:
utc_time_str = '2022-12-10 00:00:00'
# Convert user input string to datetime object in UTC timezone
utc_time = datetime.datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.UTC)

# Get California timezone
ca_tz = pytz.timezone('America/Los_Angeles')

# Convert UTC time to California local time
ca_time = utc_time.astimezone(ca_tz)

# Print California local time in YYYY-MM-DD HH:MM:SS format
print("California local time is:", ca_time.strftime('%Y-%m-%d %H:%M:%S'))

California local time is: 2022-12-09 16:00:00


In [93]:
vt

,visitdatetime,timezone,localized_time
40243,2022-12-10 00:00:00,America/Los_Angeles,16
14640,2022-12-10 00:00:11,America/New_York,19
62244,2022-12-10 00:00:19,America/Detroit,19
97446,2022-12-10 00:00:20,America/New_York,19
122350,2022-12-10 00:00:24,America/Chicago,18
...,...,...,...
60417,2022-12-10 23:59:46,America/Detroit,18
79229,2022-12-10 23:59:49,America/New_York,18
46180,2022-12-10 23:59:49,America/Los_Angeles,15
42948,2022-12-10 23:59:55,America/New_York,18


## df_imp

In [ ]:
df_imp

In [ ]:
#plot to see how many variables we should take

#curve flattens at around 50 and it becomes diminishing returns 

#can do cutoff earlier too if computation is too much

plt.plot(range(0,len(df_imp),1), df_imp['CUMSUM'])  

In [ ]:
df_second_subset[df_second_subset['checkoutthankyouflag'] == 1]

# device type

In [ ]:
X=df_second_subset['devicetype']
Y=df_second_subset['checkoutthankyouflag'] ## target

In [ ]:
X = pd.get_dummies(df_second_subset['devicetype'])
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1) # 70% training and 30% test

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

model=DecisionTreeClassifier() ## define 
#param={'criterion':['gini','entropy'],'max_depth':range(3,50)}
param={'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [3, 4, 5, 6, 7, 8, 9],
              'criterion' :['gini','entropy'],
               'splitter': ['best', 'random'],
             }
grid_search=GridSearchCV(model, param,cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)
print (grid_search.best_score_, grid_search.best_params_)

In [ ]:
# final_model=DecisionTreeClassifier()
# #final_model=DecisionTreeClassifier(**grid_search.best_params_)
# final_model.fit(Xtrain_pca, y_train)
# y_pred=final_model.predict(Xtest_pca)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
from sklearn import metrics


final_model=DecisionTreeClassifier()
#final_model=DecisionTreeClassifier(**grid_search.best_params_)
final_model.fit(X_train, y_train)
y_pred=final_model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
test=pd.DataFrame(final_model.feature_importances_)
test['checkoutdevice']=X.columns

In [ ]:
# test
# test.columns
test = test.rename(columns={0: 'rate'})
test = test.sort_values(by='rate', ascending=False)
test


In [ ]:
#sns.barplot(x='rate',y='check out categories',data=test,orient='h'
           #).set(title='categories while checking out for all devices')
import seaborn as sns


sns.set()
ax = sns.barplot(x=test.rate,y=test.checkoutdevice,orient='h')

#plt.title('Sales Data', fontsize=20)
ax.axes.set_title('checkout devices',fontsize=15)
#ax.set(title = 'Top 20 categories while checking out for all devices', fontsize=20)
# sns.set(rc={'figure.figsize':(5,5)})
#sns.barplot(x='rate',y='check out categories',data=test,orient='h').set(title='categories while checking out for all devices')

## life stage

In [ ]:
df_second_subset['prop29']

In [ ]:
X=df_second_subset['prop29']
Y=df_second_subset['checkoutthankyouflag'] ## target

In [ ]:
X = pd.get_dummies(df_second_subset['prop29'])
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1) # 70% training and 30% test

In [ ]:
# final_model=DecisionTreeClassifier()
# #final_model=DecisionTreeClassifier(**grid_search.best_params_)
# final_model.fit(Xtrain_pca, y_train)
# y_pred=final_model.predict(Xtest_pca)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
from sklearn import metrics


final_model=DecisionTreeClassifier()
#final_model=DecisionTreeClassifier(**grid_search.best_params_)
final_model.fit(X_train, y_train)
y_pred=final_model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
test=pd.DataFrame(final_model.feature_importances_)
test['prop29']=X.columns

In [ ]:
# test
# test.columns
test = test.rename(columns={0: 'rate'})
test = test.sort_values(by='rate', ascending=False)
test


In [ ]:
#sns.barplot(x='rate',y='check out categories',data=test,orient='h'
           #).set(title='categories while checking out for all devices')
import seaborn as sns


sns.set()
ax = sns.barplot(x=test.rate,y=test.prop29,orient='h')

#plt.title('Sales Data', fontsize=20)
ax.axes.set_title('checkout devices',fontsize=15)
#ax.set(title = 'Top 20 categories while checking out for all devices', fontsize=20)
# sns.set(rc={'figure.figsize':(5,5)})
#sns.barplot(x='rate',y='check out categories',data=test,orient='h').set(title='categories while checking out for all devices')